## Motivation
### What is your dataset?
The dataset used in this project comes from the data provided for the Yelp Dataset Challenge. This dataset consists of about 1.5 million users and about 200 thousand businesses from North America.  Additionally the dataset includes just under 6 million reviews, made by users of the Yelp service, to businesses. The businesses included in the dataset are both restaurants as well as businesses offering other services, such as postal delivery. 

### Why did you choose this/these particular dataset(s)?
The dataset is tremendous
### What was your goal for the end user's experience?
The purpose of this project is to investigate properties of Yelp’s Elite users. For this paper, the focus will lie on Yelp’s two primary claims about their Elite users:

Yelp states that its Elite users have high connectivity, which means that they are connected with many other users and interact often with members of their Yelp community. 

Yelp claims that its Elite users make up the “true heart of the Yelp community.” Third, Yelp claims that its users have high contribution, which means that the user has made a large impact on the site with meaningful and high-quality reviews. 

The first goal of our project is to analyze whether the above claims about Yelp’s Elite users are quantifiably valid. For this, we will specify several characteristics which we expect Elite users to have based on these claims. We will then perform analyses on Yelp’s dataset in order to determine whether these properties are truly represented among the Elite users. The secondary goal of our project is to find which properties are most indicative of Elite status on Yelp. 

The analyses for the first goal can be used for this purpose as well. This kind of information may be useful for those who are interested in becoming Elite members on Yelp. In order to become a member of the “Elite squad,” a user must go through an application process. Despite the suggestions presented above, Yelp doesn’t provide any specific criteria on exactly what characteristics a user must have to become Elite. The mystery behind the selection process for Elite users is well-documented.




## Basic stats. Let's understand the dataset better
### Write about your choices in data cleaning and preprocessing
- Mis-formatted JSON to valid JSON

In [1]:
import pandas as pd
def cleanup(k, dataset, chunk_size):
    dirty_path = 'yelp_dataset/yelp_academic_dataset_%s.json' % dataset
    clean_path = "cleaned/%s%i.json" % (dataset, k)
    dirty_file = open(dirty_path, "r")
    clean_file = open(clean_path, "w")
    
    start = chunk_size * k
    end = chunk_size * (k+1)
    
    content = ''
    i = 0
    for line in dirty_file:
        if i == end:
            break
        elif i >= start:
            s = line.replace('\n', ',\n')
            content += s
        i += 1
    if content:
        payload = '{"data" : \n[%s]}' % (content[:-2] + '\n')
        clean_file.write(payload)
    else:
        print("No more content.")

### Get all restaurants from Toronto

In [ ]:
# Load in JSON data, convert to a dataframe
json_data = list(pd.read_json('cleaned/business.json').data)
df = pd.DataFrame(json_data)

# Restaurants will contain the keywords 'restaurant' 
# and/or 'food' in the 'category' attribute.
keywords = ['restaurant', 'food']
idx = df.categories.str.lower().str.contains("|".join(keywords)).fillna(False)
rest = df[idx]

# Drop attributes irrelevant to the analysis
rest = rest.drop(['attributes', 'categories', 'address', 'neighborhood'], axis=1)

# Save dataset to CSV
rest.to_csv('cleaned_csv/restaurants.csv')

In [ ]:
biz = pd.read_csv('cleaned_csv/restaurants.csv').drop(['is_open', 'hours', 'Unnamed: 0'], axis=1)

### Get all reviews from Toronto

### Get all users in the Toronto reviews

In [ ]:
for k in range(20):
    chunk_size = 100 * 1000
    cleanup(k, 'user', chunk_size)
    print('Iteration', k, 'done')

In [ ]:
N = 16
df_matrix = [None] * N
for i in range(N):
    path = "cleaned/user%i.json" % i
    df_matrix[i] = pd.DataFrame(list(pd.read_json(path).data))

DF = pd.concat(df_matrix)
reviews = pd.read_csv('toronto/reviews_all_time.csv')

# Filter out users not in the Toronto reviews
toronto_users = DF[DF.user_id.isin(reviews.user_id)]

# Throw out unused columns, to save space
toronto_users = toronto_users.drop(['compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
     'funny', 'fans'], axis=1)

# Save to CSV
toronto_users.to_csv('toronto/toronto_users.csv', index=False)

In [ ]:
# Get all restaurants in Toronto
biz = pd.read_csv('cleaned_csv/business.csv').drop(['is_open', 'hours', 'Unnamed: 0'], axis=1)
biz.city = biz.city.str.lower()
biz_toronto = biz[biz.city == 'toronto']
print("Restaurants in Toronto: ", biz_toronto.shape[0])

In [ ]:
# Make 
n = 6
df_list = [None] * n
for i in range(n):
    path = "cleaned/review%i.json" % i
    df_list[i] = pd.DataFrame(list(pd.read_json(path).data))    

DF = pd.concat(df_list)
reviews = pd.read_csv('toronto/reviews_all_time.csv')

In [ ]:
- JSON to DataFrame, to CSV (takes up less space)
- Select only those businesses which are located in Toronto
- Filter out non-restaurants
- Use the dataframe of Toronto restaurants to identify reviews made by a user, reviewing a Toronto restaurants
- Use the list of Toronto reviews to filter out users who have not contributed to the Toronto Yelp reviews

### Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)
- Reviews: ~6 million
- Users: Many
- Businesses: ~200,000

For this project the restaurants in Toronto were the main focus, as Toronto is a big city with more than a sufficient amount of data to perform a serious analysis, but small enough for various graph algorithms to be carried out. The users considered in this project were all the users who left a review on a business in Toronto.

- Period: March 1st 2008 to August 1st 2018
- Reviews: ~380,000
- Users: ~85,000
- Elite users hereof: ~7,500
- Restaurants: ~10,000

This is analogous to the paper which had 250k nodes and 950k edges, where both are just scaled up by around a factor of 2.5!



## Tools, theory and analysis. Describe the process of theory to insight
### Talk about how you've worked with text, including regular expressions, unicode, etc.
### Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.
#### Modelling the network
The Toronto Yelp review network was modelled as an undirected graph, containing user nodes where the edges between two user nodes represent the fact two users have reviewed the same restaurant. 

#### Most connected subcomponent
Detecting how important the elite users were for the network was done by deleting them one by one from the graph, and then watching how the largest connected subgraph shrinks. The elite users were deleted based on their degree centrality.
### How did you use the tools to understand your dataset?



## Discussion. Think critically about your creation
### What went well?
### What is still missing? What could be improved?